In [49]:
import pandas as pd 
import numpy as np
from pathlib import Path
import glob 
import importlib
from functions import open_workout_file, get_all_workout_files, build_exercise_dataframe, extract_exercise_sets, extract_set_data
from functions import calculate_weighted_set_volume, extract_bodyweight_volume, extract_volume, get_workout_name, get_workout_date
import functions
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [50]:
# import workout files 
workout_files = get_all_workout_files()
print(f"Found {len(workout_files)} workout files")

Found 16 workout files


In [51]:
line = open_workout_file(workout_files[11])
print("First line of first workout file:", line[0])
print("Fifth line of first workout file:", line[5])

First line of first workout file: Upper MP B

Fifth line of first workout file: Set 2 : 50 kg  x 8



In [52]:
calculate_weighted_set_volume(line[5])

400.0

In [53]:
total = extract_volume(workout_files[0], bodyweight=80)
print(f"Total volume: {total} kg")

Total volume: 224.0 kg


In [54]:
for file in workout_files:
    total = extract_volume(file, bodyweight=80)
    print(f"{file}: {total} kg")

../data/raw/workouts/26_09_25.txt: 224.0 kg
../data/raw/workouts/25_10_25.txt: 1322.0 kg
../data/raw/workouts/09_10_25.txt: 2204.0 kg
../data/raw/workouts/22_10_25.txt: 4390.0 kg
../data/raw/workouts/11_10_25.txt: 7514.0 kg
../data/raw/workouts/05_10_25.txt: 5338.0 kg
../data/raw/workouts/24_10_25.txt: 2667.0 kg
../data/raw/workouts/21_10_25.txt: 1945.0 kg
../data/raw/workouts/27_10_25.txt: 2069.0 kg
../data/raw/workouts/06_10_25.txt: 9711.0 kg
../data/raw/workouts/24_09_25.txt: 5653.0 kg
../data/raw/workouts/12_10_25.txt: 6117.0 kg
../data/raw/workouts/15_10_25.txt: 11275.0 kg
../data/raw/workouts/07_10_25.txt: 4451.0 kg
../data/raw/workouts/25_09_25.txt: 5469.5 kg
../data/raw/workouts/28_09_25.txt: 4935.0 kg


In [55]:
for file in workout_files:
    print(f"Processing: {file}")
    total = extract_volume(file, bodyweight=80)
    print(f"  Total: {total} kg")

Processing: ../data/raw/workouts/26_09_25.txt
  Total: 224.0 kg
Processing: ../data/raw/workouts/25_10_25.txt
  Total: 1322.0 kg
Processing: ../data/raw/workouts/09_10_25.txt
  Total: 2204.0 kg
Processing: ../data/raw/workouts/22_10_25.txt
  Total: 4390.0 kg
Processing: ../data/raw/workouts/11_10_25.txt
  Total: 7514.0 kg
Processing: ../data/raw/workouts/05_10_25.txt
  Total: 5338.0 kg
Processing: ../data/raw/workouts/24_10_25.txt
  Total: 2667.0 kg
Processing: ../data/raw/workouts/21_10_25.txt
  Total: 1945.0 kg
Processing: ../data/raw/workouts/27_10_25.txt
  Total: 2069.0 kg
Processing: ../data/raw/workouts/06_10_25.txt
  Total: 9711.0 kg
Processing: ../data/raw/workouts/24_09_25.txt
  Total: 5653.0 kg
Processing: ../data/raw/workouts/12_10_25.txt
  Total: 6117.0 kg
Processing: ../data/raw/workouts/15_10_25.txt
  Total: 11275.0 kg
Processing: ../data/raw/workouts/07_10_25.txt
  Total: 4451.0 kg
Processing: ../data/raw/workouts/25_09_25.txt
  Total: 5469.5 kg
Processing: ../data/raw/w

In [56]:
name = get_workout_name(workout_files[0])
print(name)

Sep 26 Workout


In [57]:
date = get_workout_date(workout_files[0])
print(date)

2025-09-26


### Create dataframe for workout data

In [58]:
# Create lists to store data
dates = []
names = []
volumes = []

# Loop through all workout files
for file in workout_files:
    date = get_workout_date(file)
    name = get_workout_name(file)
    volume = extract_volume(file, bodyweight=80)
    
    dates.append(date)
    names.append(name)
    volumes.append(volume)

# Create DataFrame
workout_df = pd.DataFrame({
    'date': dates,
    'workout_name': names,
    'total_volume': volumes
})

print(workout_df)

          date               workout_name  total_volume
0   2025-09-26             Sep 26 Workout         224.0
1   2025-10-25  Focus: Chest, Delts, Arms        1322.0
2   2025-10-09              Oct 9 Workout        2204.0
3   2025-10-22  Focus: Chest, Delts, Arms        4390.0
4   2025-10-11               BACK & DELTS        7514.0
5   2025-10-05                 Upper MP B        5338.0
6   2025-10-24  Focus: Chest, Delts, Arms        2667.0
7   2025-10-21               BACK & DELTS        1945.0
8   2025-10-27  Focus: Chest, Delts, Arms        2069.0
9   2025-10-06              Oct 6 Workout        9711.0
10  2025-09-24                 Upper MP A        5653.0
11  2025-10-12                 Upper MP B        6117.0
12  2025-10-15             Oct 15 Workout       11275.0
13  2025-10-07                 Upper MP A        4451.0
14  2025-09-25               BACK & DELTS        5469.5
15  2025-09-28                 Upper MP B        4935.0


In [59]:
# save to csv
workout_df.to_csv('../data/processed/workout_summary.csv', index=False)
print("Workout summary saved to ../data/processed/workout_summary.csv")

Workout summary saved to ../data/processed/workout_summary.csv


In [68]:
# Define which exercises you want to track
my_exercises = [
    "Bench Press (Barbell)",
    "Overhead Tricep Extension (Cable)", 
    "Romanian Deadlift (Barbell)",
    "Lateral Raise (Dumbbell)",
    "Seated Overhead Press (Dumbbell)",
]

# Build the dataframe
exercise_df = build_exercise_dataframe(my_exercises, bodyweight=80)

# Look at it!
exercise_df.head()

,date,exercise,set_number,weight,reps,volume
0,2025-09-26,Lateral Raise (Dumbbell),1,2.0,8,16.0
1,2025-09-26,Lateral Raise (Dumbbell),2,2.0,11,22.0
2,2025-09-26,Lateral Raise (Dumbbell),3,2.0,13,26.0
3,2025-09-26,Lateral Raise (Dumbbell),4,4.0,13,52.0
4,2025-09-26,Lateral Raise (Dumbbell),5,6.0,10,60.0


In [66]:
exercise_df.max()

date                  2025-10-27 00:00:00
exercise      Romanian Deadlift (Barbell)
set_number                              6
weight                               70.0
reps                                   17
volume                              720.0
dtype: object

In [62]:
# Get max weight lifted for Bench Press (Barbell)
print(exercise_df[exercise_df['exercise'] == 'Bench Press (Barbell)']['weight'].max())

60.0


In [72]:
# get PR for each exercise
exercise_df.groupby('exercise')[['weight', 'volume', 'reps', 'date']].max()

,weight,volume,reps,date
exercise,,,,
Bench Press (Barbell),60.0,500.0,10,2025-10-27
Lateral Raise (Dumbbell),8.0,72.0,17,2025-10-24
Overhead Tricep Extension (Cable),35.0,275.0,13,2025-10-27
Romanian Deadlift (Barbell),70.0,720.0,12,2025-10-22
Seated Overhead Press (Dumbbell),24.0,264.0,13,2025-10-27
